In [1]:
import torch
import torch.nn as nn
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator
from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import Dataset, DataLoader

Input:
- Question + <'pad'> + context

Output:
- The start index and end index of the extracted information from context

In [2]:
qa_dataset = [
    {
        'context': 'My name is Thang and I am from Vietnam',
        'question': 'What is my name?',
        'answer': 'Thang'
    },
    {
        'context': 'I love painting and my favorite artist is Vincent Van Gough',
        'question': 'What is my favorite activity?',
        'answer' : 'painting'
    },
    {
        'context' : 'I am studying Computer Science at Nanyang Technological University',
        'question' : 'What am I studying?',
        'answer' : 'Computer Science'
    }
]

In [3]:
#define tokenizer
tokenizer = get_tokenizer('basic_english')

text = 'I love cats'
tokenizer(text)

['i', 'love', 'cats']

In [4]:
def yield_tokens(data):
    for item in data:
        yield(tokenizer('<cls>' + item['context'] + '<sep>' + item['question']))

#create vocab
vocab = build_vocab_from_iterator(
    yield_tokens(qa_dataset),
    specials=['<unk>', '<pad>', '<bos>', '<eos>', '<sep>', '<cls>']
)
vocab.set_default_index(vocab['<unk>'])
vocab.get_stoi()

{'<cls>': 5,
 '<sep>': 4,
 'vietnam<sep>what': 31,
 'technological': 27,
 '<bos>': 2,
 '<unk>': 0,
 '?': 7,
 'artist': 18,
 '<eos>': 3,
 '<pad>': 1,
 'is': 6,
 'my': 9,
 'am': 8,
 '<cls>i': 10,
 'name': 14,
 'and': 11,
 'favorite': 12,
 'i': 13,
 'studying': 15,
 'nanyang': 24,
 '<cls>my': 16,
 'activity': 17,
 'at': 19,
 'computer': 20,
 'from': 21,
 'gough<sep>what': 22,
 'love': 23,
 'painting': 25,
 'science': 26,
 'thang': 28,
 'university<sep>what': 29,
 'van': 30,
 'vincent': 32}

In [20]:
PAD_IDX = 1
def pad_and_truncate(input_ids, max_seq_len):
    if len(input_ids) > max_seq_len:
        input_ids = input_ids[:max_seq_len]
    elif len(input_ids) < max_seq_len:
        input_ids += [PAD_IDX] * (max_seq_len - len(input_ids))

    return input_ids

In [21]:
MAX_SEQ_LENGTH = 22
def vectorize(question, context, answer, max_seq_len):
    input_text = question + '<sep>' + context
    input_ids = [vocab[token] for token in tokenizer(input_text)]
    input_ids = pad_and_truncate(input_ids, MAX_SEQ_LENGTH)

    answer_ids = [vocab[token] for token in tokenizer(answer)]
    start_positions = input_ids.index(answer_ids[0])
    end_positions = start_positions + len(answer_ids) - 1

    input_ids = torch.tensor(input_ids, dtype=torch.long)
    start_positions = torch.tensor(start_positions, dtype=torch.long)
    end_positions = torch.tensor(end_positions, dtype=torch.long)

    return input_ids, start_positions, end_positions 

In [22]:
class QADataset(nn.Module):
    def __init__(self, data):
        self.data = data

    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        item = self.data[idx]
        question_text = item['question']
        context_text = item['context']
        answer_text = item['answer']

        input_ids, start_positions, end_positions = vectorize(
            question_text, context_text, answer_text, 22
        )

        return input_ids, start_positions, end_positions

In [23]:
def decode(input_ids):
    return ' '.join([vocab.lookup_token(token) for token in input_ids])

In [24]:
train_dataset = QADataset(qa_dataset)
train_loader = DataLoader(train_dataset, batch_size=2, shuffle=True)

for batch in train_loader:
    q, c, a = batch
    print(q, c, a)
   

tensor([[ 0,  8, 13, 15,  7,  0,  8, 15, 20, 26, 19, 24, 27,  0,  1,  1,  1,  1,
          1,  1,  1,  1],
        [ 0,  6,  9, 14,  7,  0, 14,  6, 28, 11, 13,  8, 21,  0,  1,  1,  1,  1,
          1,  1,  1,  1]]) tensor([8, 8]) tensor([9, 8])
tensor([[ 0,  6,  9, 12, 17,  7,  0, 23, 25, 11,  9, 12, 18,  6, 32, 30,  0,  1,
          1,  1,  1,  1]]) tensor([8]) tensor([8])


In [49]:
class TokenAndPostitionEmbedding(nn.Module):
    def __init__(self, vocab_size, embed_dim, max_length):
        super().__init__()
        self.word_embed = nn.Embedding(num_embeddings=vocab_size,
                                       embedding_dim=embed_dim)
        self.pos_embed = nn.Embedding(num_embeddings=max_length,
                                      embedding_dim=embed_dim)
    
    def forward(self, x):
        N, seq_len = x.size()
        positions = torch.arange(0, seq_len).expand(N, seq_len)
        output_1 = self.word_embed(x)
        output_2 = self.pos_embed(positions)
        output = output_1 + output_2
        return output

In [63]:
class TransformerBlock(nn.Module):
    def __init__(self, embed_dim, num_heads, ff_dim, output_dim, dropout=0.1):
        super().__init__()
        self.attn = nn.MultiheadAttention(embed_dim=embed_dim,
                                          num_heads=num_heads,
                                          batch_first=True
                                        )
        self.ffn = nn.Sequential(
            nn.Linear(in_features=embed_dim, out_features=ff_dim, bias=True),
            nn.ReLU(),
            nn.Linear(in_features=ff_dim, out_features=embed_dim, bias=True)
        )
        self.layernorm1 = nn.LayerNorm(normalized_shape=embed_dim)
        self.layernorm2 = nn.LayerNorm(normalized_shape=embed_dim)
        self.dropout1 = nn.Dropout(p=dropout)
        self.dropout2 = nn.Dropout(p=dropout)
        self.linear_out = nn.Linear(in_features=embed_dim,
                                    out_features=output_dim)

    def forward(self, query, key, value):
        attn_output, _ = self.attn(query, key, value)
        attn_output = self.dropout1(attn_output)
        out_1 = self.layernorm1(query + attn_output)
        ffn_out = self.ffn(out_1)
        ffn_out = self.dropout2(ffn_out)
        out_2 = self.layernorm2(ffn_out + out_1)
        out_2 = self.linear_out(out_2)
        return out_2


In [64]:
model = TransformerBlock(embed_dim=32, num_heads=1, ff_dim=128, output_dim=128)
model(torch.randn(1, 32), torch.randn(1, 32), torch.randn(1, 32)).shape

torch.Size([1, 128])

In [68]:
# Model: transformer
class QAModel(nn.Module):
    def __init__(self, 
                 vocab_size, embedding_dim, n_heads,
                 ff_dim, max_len):
        super().__init__()
        self.input_embedding = TokenAndPostitionEmbedding(
            vocab_size, embedding_dim, max_len
        )

        self.transformer = TransformerBlock(embedding_dim, n_heads, ff_dim=64, output_dim=ff_dim)
        self.start_linear = nn.Linear(ff_dim, 1)
        self.end_linear = nn.Linear(ff_dim, 1)

    def forward(self, text):
        input_embedded = self.input_embedding(text)
        transformer_out = self.transformer(input_embedded, input_embedded, input_embedded)
        start_logits = self.start_linear(transformer_out).squeeze(-1)
        end_logits = self.end_linear(transformer_out).squeeze(-1)
        return start_logits, end_logits

In [95]:
#Model parameters
EMBEDDING_DIM = 32
HIDDEN_SIZE = 128
VOCAB_SIZE = len(vocab)
N_HEADS = 1


model = QAModel(
    VOCAB_SIZE, EMBEDDING_DIM, N_HEADS, HIDDEN_SIZE, MAX_SEQ_LENGTH
)

input_context = torch.randint(0, 10, size=(1, 10))
model.eval()
with torch.no_grad():
    start_logits, end_logits = model(input_context)

#print(start_logits.shape)

In [96]:
LR = 1e-3
EPOCHS = 20
optimizer = torch.optim.Adam(
    model.parameters(), lr=LR
)
criterion = nn.CrossEntropyLoss()

model.train()
for _ in range(EPOCHS):
    for idx, (input_ids, start_positions, end_positions) in enumerate(train_loader):
        optimizer.zero_grad()
        start_logits, end_logits = model(input_ids)
        start_loss = criterion(start_logits, start_positions)
        end_loss = criterion(end_logits, end_positions)
        loss = start_loss + end_loss
        loss.backward()
        optimizer.step()
        print(loss.item())

6.281930923461914
5.625787734985352
5.431855201721191
5.7655229568481445
5.200865745544434
4.5978851318359375
4.624580383300781
4.085083484649658
4.106523513793945
3.561677932739258
3.59871768951416
3.0926315784454346
2.9918417930603027
2.998030662536621
2.657653331756592
2.2859842777252197
1.9957795143127441
2.00809907913208
1.5538716316223145
1.5510176420211792
1.3236503601074219
0.7917777299880981
0.7757298946380615
1.1117939949035645
0.7248416543006897
0.5187174081802368
0.5942517518997192
0.22171202301979065
0.41816240549087524
0.20695018768310547
0.17814938724040985
0.4796469211578369
0.27517473697662354
0.12485484778881073
0.18407566845417023
0.0612393394112587
0.12700507044792175
0.044127076864242554
0.12990613281726837
0.02366236224770546


In [97]:
model.eval()
with torch.no_grad():
    sample = qa_dataset[1]
    context, question, answer = sample.values()
    input_ids, start_position, end_position = vectorize(context, question, answer, MAX_SEQ_LENGTH)
    input_ids = input_ids.unsqueeze(0)
    start_logits, end_logits = model(input_ids)

    offset = len(tokenizer(question)) + 2
    start_position = torch.argmax(start_logits, dim=1).numpy()[0]
    end_position = torch.argmax(end_logits, dim=1).numpy()[0]

    start_position -= offset
    end_position -= offset

    start_position = max(start_position, 0)
    end_position = min(end_position, len(tokenizer(context)) - 1)

    if end_position >= start_position:
        #extract the predicted answer span
        context_tokens = tokenizer(context)
        predicted_answer_tokens = context_tokens[start_position:end_position+1]
        predicted_answer = ' '.join(predicted_answer_tokens)
        print(context)
        print(question)
        print(predicted_answer)

I love painting and my favorite artist is Vincent Van Gough
What is my favorite activity?
i love


Chắc thằng cu này đòi data lớn rồi :)) cay thật